# Pipeline

In [2]:
import os

OPENAI_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_KEY:
    raise ValueError("OpenAI API key not found in environment variables")

In [1]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [2]:
full_template = """{introduction}

{example}

{start}"""
full_prompt = PromptTemplate.from_template(full_template)

In [3]:
introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

In [4]:
example_template = """Here's an example of an interaction:

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

In [5]:
start_template = """Now, do this for real!

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

In [6]:
input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt),
]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=input_prompts
)
pipeline_prompt.input_variables

['example_q', 'person', 'example_a', 'input']

In [7]:
print(
    pipeline_prompt.format(
        person="Elon Musk",
        example_q="What's your favorite car?",
        example_a="Tesla",
        input="What's your favorite social media site?",
    )
)

pipeline_prompt

You are impersonating Elon Musk.

Here's an example of an interaction:

Q: What's your favorite car?
A: Tesla

Now, do this for real!

Q: What's your favorite social media site?
A:


PipelinePromptTemplate(input_variables=['example_q', 'person', 'example_a', 'input'], final_prompt=PromptTemplate(input_variables=['example', 'introduction', 'start'], template='{introduction}\n\n{example}\n\n{start}'), pipeline_prompts=[('introduction', PromptTemplate(input_variables=['person'], template='You are impersonating {person}.')), ('example', PromptTemplate(input_variables=['example_a', 'example_q'], template="Here's an example of an interaction:\n\nQ: {example_q}\nA: {example_a}")), ('start', PromptTemplate(input_variables=['input'], template='Now, do this for real!\n\nQ: {input}\nA:'))])

In [8]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
chain = LLMChain(llm=model, prompt=pipeline_prompt)

# 입력 데이터에 필요한 키들을 포함
input_data = {
    'example_q': "What's your favorite car?",  # 예시 질문
    'example_a': "Tesla",                      # 예시 대답
    'input': "What's your favorite social media site?",  # 처리할 실제 질문
    'person': "Elon Musk"                      # 모방할 인물
}

chain.run(input_data)


c:\Users\Acorn\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Twitter.'

# Partial with strings

In [35]:
from langchain.prompts import PromptTemplate


prompt = PromptTemplate(template="{foo}{bar}", input_variables=["foo", "bar"])
print(prompt)
partial_prompt = prompt.partial(foo="foo")
print(partial_prompt)
print(partial_prompt.format(bar="baz"))

input_variables=['bar', 'foo'] template='{foo}{bar}'
input_variables=['bar'] partial_variables={'foo': 'foo'} template='{foo}{bar}'
foobaz


In [36]:
prompt = PromptTemplate(
    template="{foo}{bar}", input_variables=["bar"], partial_variables={"foo": "foo"}
)
print(prompt.format(bar="baz"))

foobaz


# Partial with functions

In [37]:
from datetime import datetime


def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")

In [38]:
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective", "date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))

Tell me a funny joke about the day 01/17/2024, 09:58:07


In [48]:
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}", input_variables=["adjective"], partial_variables={"date": lambda:_get_datetime}
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))

Tell me a funny joke about the day 01/17/2024, 10:09:19


In [40]:
partial_prompt.input_variables

['adjective']

In [41]:
model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
chain = LLMChain(llm=model, prompt=partial_prompt)

input_data = {
    "adjective" : "sad"
}
chain.run(input_data)

"Why did the clock feel so lonely on 01/17/2024, 10:00:55?\n\nBecause it realized that time was ticking away, but it couldn't turn back to the moments it cherished the most."